In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Adjust display settings for wider output
pd.set_option('display.width', 300)  # Increase the display width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 10)  # Limit rows displayed for clarity


consumption = pd.read_csv("/Users/olaoluwatunmise/Germany-Energy-Price-Forecast/PowerCast-Datasets/Consumption/Actual_consumption_202301010000_202503050000_Quarterhour.csv", delimiter=';')

print(consumption.head())

             Start date              End date Total (grid load) [MWh] Original resolutions Residual load [MWh] Original resolutions Hydro pumped storage [MWh] Original resolutions
0  Jan 1, 2023 12:00 AM  Jan 1, 2023 12:15 AM                                     9,673.00                                 1,842.50                                          494.00
1  Jan 1, 2023 12:15 AM  Jan 1, 2023 12:30 AM                                     9,593.50                                 1,691.50                                          502.50
2  Jan 1, 2023 12:30 AM  Jan 1, 2023 12:45 AM                                     9,562.00                                 1,442.50                                          561.00
3  Jan 1, 2023 12:45 AM   Jan 1, 2023 1:00 AM                                     9,517.50                                 1,598.50                                          519.25
4   Jan 1, 2023 1:00 AM   Jan 1, 2023 1:15 AM                                     9,433.25          

# Exploratory Data Analysis

### Clean the column names

In [14]:
# Clean the column names
consumption.columns = consumption.columns.str.replace(r'[^\w\s]', '', regex=True)  # Remove special characters
consumption.columns = consumption.columns.str.strip()  # Remove leading/trailing spaces
consumption.columns = consumption.columns.str.replace('Original resolutions', '', regex=False)  # Remove redundant words
# Remove "MWh" from the column names
consumption.columns = consumption.columns.str.replace(' MWh ', '', regex=False)
# Replace spaces between words with underscores for consistency
consumption.columns = consumption.columns.str.replace(' ', '_', regex=False)

# Replace multiple consecutive underscores with a single one (in case multiple spaces exist)
consumption.columns = consumption.columns.str.replace('_+', '_', regex=True)

# Ensure all column names are lowercase (optional, for consistency)
consumption.columns = consumption.columns.str.lower()

#new_consumption = consumption

print(consumption.columns)
print(consumption)

Index(['start_date', 'end_date', 'total_grid_load', 'residual_load', 'hydro_pumped_storage'], dtype='object')
                 start_date              end_date total_grid_load residual_load hydro_pumped_storage
0      Jan 1, 2023 12:00 AM  Jan 1, 2023 12:15 AM        9,673.00      1,842.50               494.00
1      Jan 1, 2023 12:15 AM  Jan 1, 2023 12:30 AM        9,593.50      1,691.50               502.50
2      Jan 1, 2023 12:30 AM  Jan 1, 2023 12:45 AM        9,562.00      1,442.50               561.00
3      Jan 1, 2023 12:45 AM   Jan 1, 2023 1:00 AM        9,517.50      1,598.50               519.25
4       Jan 1, 2023 1:00 AM   Jan 1, 2023 1:15 AM        9,433.25      1,325.50               301.00
...                     ...                   ...             ...           ...                  ...
76219  Mar 4, 2025 10:45 PM  Mar 4, 2025 11:00 PM               -             -                    -
76220  Mar 4, 2025 11:00 PM  Mar 4, 2025 11:15 PM               -             -   

In [15]:
print(consumption.isna().sum())

print(consumption.describe())

print(consumption.dtypes)

start_date              0
end_date                0
total_grid_load         0
residual_load           0
hydro_pumped_storage    0
dtype: int64
                  start_date              end_date total_grid_load residual_load hydro_pumped_storage
count                  76224                 76224           76224         76224                76224
unique                 76216                 76214           31683         40596                 6366
top     Oct 29, 2023 2:30 AM  Oct 29, 2023 2:45 AM               -             -                 1.25
freq                       2                     2              17            17                 1150
start_date              object
end_date                object
total_grid_load         object
residual_load           object
hydro_pumped_storage    object
dtype: object


In [16]:
# Load raw data
#consumption = pd.read_csv("Datasets/Actual_consumption_Quarterhour.csv")
print("Rows with '-' in total_grid_load:", (consumption["total_grid_load"] == "-").sum())
print("Rows with '-' in residual_load:", (consumption["residual_load"] == "-").sum())
print("Rows with '-' in hydro_pumped_storage:", (consumption["hydro_pumped_storage"] == "-").sum())



Rows with '-' in total_grid_load: 17
Rows with '-' in residual_load: 17
Rows with '-' in hydro_pumped_storage: 17


In [17]:
# Replace "-" with NaN
consumption.replace("_", np.nan, inplace=True)

#Convert Columns to Numeric
consumption["total_grid_load"] = pd.to_numeric(consumption["total_grid_load"], errors="coerce")
consumption["residual_load"] = pd.to_numeric(consumption["residual_load"], errors="coerce")
consumption["hydro_pumped_storage"] = pd.to_numeric(consumption["hydro_pumped_storage"], errors="coerce")


print(consumption)

                 start_date              end_date  total_grid_load  residual_load  hydro_pumped_storage
0      Jan 1, 2023 12:00 AM  Jan 1, 2023 12:15 AM              NaN            NaN                494.00
1      Jan 1, 2023 12:15 AM  Jan 1, 2023 12:30 AM              NaN            NaN                502.50
2      Jan 1, 2023 12:30 AM  Jan 1, 2023 12:45 AM              NaN            NaN                561.00
3      Jan 1, 2023 12:45 AM   Jan 1, 2023 1:00 AM              NaN            NaN                519.25
4       Jan 1, 2023 1:00 AM   Jan 1, 2023 1:15 AM              NaN            NaN                301.00
...                     ...                   ...              ...            ...                   ...
76219  Mar 4, 2025 10:45 PM  Mar 4, 2025 11:00 PM              NaN            NaN                   NaN
76220  Mar 4, 2025 11:00 PM  Mar 4, 2025 11:15 PM              NaN            NaN                   NaN
76221  Mar 4, 2025 11:15 PM  Mar 4, 2025 11:30 PM              N

In [18]:
print(consumption.dtypes)
print(consumption.columns)
print(consumption.isna().sum())


start_date               object
end_date                 object
total_grid_load         float64
residual_load           float64
hydro_pumped_storage    float64
dtype: object
Index(['start_date', 'end_date', 'total_grid_load', 'residual_load', 'hydro_pumped_storage'], dtype='object')
start_date                  0
end_date                    0
total_grid_load         76224
residual_load           74442
hydro_pumped_storage    10158
dtype: int64


In [19]:
print(consumption.isna().sum())  # Shows count of NaN values per column
print(consumption[consumption.isna().any(axis=1)])  # Displays rows with NaN values
#prices["northern_italy"] = prices["northern_italy"].fillna(prices["northern_italy"].mean())

print(consumption)

start_date                  0
end_date                    0
total_grid_load         76224
residual_load           74442
hydro_pumped_storage    10158
dtype: int64
                 start_date              end_date  total_grid_load  residual_load  hydro_pumped_storage
0      Jan 1, 2023 12:00 AM  Jan 1, 2023 12:15 AM              NaN            NaN                494.00
1      Jan 1, 2023 12:15 AM  Jan 1, 2023 12:30 AM              NaN            NaN                502.50
2      Jan 1, 2023 12:30 AM  Jan 1, 2023 12:45 AM              NaN            NaN                561.00
3      Jan 1, 2023 12:45 AM   Jan 1, 2023 1:00 AM              NaN            NaN                519.25
4       Jan 1, 2023 1:00 AM   Jan 1, 2023 1:15 AM              NaN            NaN                301.00
...                     ...                   ...              ...            ...                   ...
76219  Mar 4, 2025 10:45 PM  Mar 4, 2025 11:00 PM              NaN            NaN                   NaN
76220

In [20]:
print(consumption.index.duplicated().sum())  # Count duplicate timestamps
print(consumption[consumption.index.duplicated(keep=False)])  # Show duplicates


0
Empty DataFrame
Columns: [start_date, end_date, total_grid_load, residual_load, hydro_pumped_storage]
Index: []


In [21]:
# 1. Convert "Start date" to datetime (handle errors)
consumption["start_date"] = pd.to_datetime(
    consumption["start_date"], 
    format="%b %d, %Y %I:%M %p",  # Add explicit format to avoid warnings
    errors='coerce'
)

# 2. Drop "End date" column early
consumption.drop(columns=["end_date"], inplace=True)

# 3. Set datetime column as index FIRST
consumption.set_index("start_date", inplace=True)

# 4. Now convert remaining columns to numeric (index is already datetime)
consumption = consumption.apply(pd.to_numeric, errors='coerce')
#consumption.iloc[:, :] = consumption.iloc[:, :].apply(pd.to_numeric, errors='coerce')

#Aggregate Duplicates Before Resampling
consumption = consumption.groupby(consumption.index).mean()  # or .sum(), .max()

# 5. Resample using the DatetimeIndex
#consumption_hourly = consumption.resample('1h').mean()
#consumption_hourly = consumption.resample('1h').ffill()  # Forward fill missing values
consumption_hourly = consumption.resample('1h').sum()


# 6. Reset index if needed (optional)
consumption_hourly.reset_index(inplace=True)

print(consumption_hourly.head())

           start_date  total_grid_load  residual_load  hydro_pumped_storage
0 2023-01-01 00:00:00              0.0            0.0               2076.75
1 2023-01-01 01:00:00              0.0            0.0               1767.75
2 2023-01-01 02:00:00              0.0         1738.0               3050.25
3 2023-01-01 03:00:00              0.0            0.0               3704.00
4 2023-01-01 04:00:00              0.0            0.0               3800.75


In [22]:
print(consumption.index.duplicated().sum())  # Count duplicate timestamps
print(consumption[consumption.index.duplicated(keep=False)])  # Show duplicates


0
Empty DataFrame
Columns: [total_grid_load, residual_load, hydro_pumped_storage]
Index: []


In [23]:
print(consumption_hourly.dtypes)
print(consumption_hourly.columns)

start_date              datetime64[ns]
total_grid_load                float64
residual_load                  float64
hydro_pumped_storage           float64
dtype: object
Index(['start_date', 'total_grid_load', 'residual_load', 'hydro_pumped_storage'], dtype='object')


In [24]:
print(consumption_hourly.isna().sum())  # Shows count of NaN values per column
print(consumption_hourly[consumption_hourly.isna().any(axis=1)])  # Displays rows with NaN values
#prices["northern_italy"] = prices["northern_italy"].fillna(prices["northern_italy"].mean())

print(consumption_hourly)

start_date              0
total_grid_load         0
residual_load           0
hydro_pumped_storage    0
dtype: int64
Empty DataFrame
Columns: [start_date, total_grid_load, residual_load, hydro_pumped_storage]
Index: []
               start_date  total_grid_load  residual_load  hydro_pumped_storage
0     2023-01-01 00:00:00              0.0            0.0               2076.75
1     2023-01-01 01:00:00              0.0            0.0               1767.75
2     2023-01-01 02:00:00              0.0         1738.0               3050.25
3     2023-01-01 03:00:00              0.0            0.0               3704.00
4     2023-01-01 04:00:00              0.0            0.0               3800.75
...                   ...              ...            ...                   ...
19051 2025-03-04 19:00:00              0.0            0.0                  1.50
19052 2025-03-04 20:00:00              0.0            0.0                  0.00
19053 2025-03-04 21:00:00              0.0            0.0    